## Baseline

https://www.kaggle.com/code/pxcai666/catboost-lightgbm-ensemble/notebook

### Configuration

In [1]:
! pip install -U polars

In [2]:
import sys
from pathlib import Path
import subprocess
import os
import gc
from glob import glob

import numpy as np
import pandas as pd
import polars as pl
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool
import lightgbm as lgb

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
ROOT = "/kaggle/input/home-credit-credit-risk-model-stability"

TRAIN_DIR = os.path.join(ROOT, "parquet_files", "train")
TEST_DIR = os.path.join(ROOT, "parquet_files", "test")

In [4]:
device = "gpu"

### Def utility classes

In [5]:
class Pipeline:
    def set_table_dtypes(df: pl.DataFrame):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"] or col.endswith("D"):
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col.endswith(("P", "A")):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col.endswith("M"):
                df = df.with_columns(pl.col(col).cast(pl.String))

        return df

    def handle_dates(df: pl.DataFrame):
        """日付関連の列を処理

        - date_decisionからの日数差分を取得
        """
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))  #!!?
                df = df.with_columns(pl.col(col).dt.total_days()) # t - t-1
        df = df.drop("date_decision", "MONTH")
        return df

    def filter_cols(df: pl.DataFrame):
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].is_null().mean()
                if isnull > 0.7:
                    df = df.drop(col)

        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.Utf8):
                freq = df[col].n_unique()
                if (freq == 1) | (freq > 200):
                    df = df.drop(col)

        return df

In [6]:

class Aggregator:
    @staticmethod
    def get_exprs(df):
        exprs = (
            Aggregator.num_expr(df)
            + Aggregator.date_expr(df)
            + Aggregator.str_expr(df)
            + Aggregator.other_expr(df)
            + Aggregator.count_expr(df)
        )

        return exprs

    @staticmethod
    def num_expr(df):
        cols = [col for col in df.columns if col.endswith(("P", "A"))]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]

        return expr_max + expr_last + expr_mean

    @staticmethod
    def date_expr(df):
        cols = [col for col in df.columns if col.endswith("D")]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]

        return expr_max + expr_last + expr_mean

    @staticmethod
    def str_expr(df):
        cols = [col for col in df.columns if col.endswith("M")]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        #expr_count = [pl.count(col).alias(f"count_{col}") for col in cols]

        return expr_max + expr_last

    @staticmethod
    def other_expr(df):
        cols = [col for col in df.columns if col.endswith(("T", "L"))]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]

        return expr_max + expr_last


    @staticmethod
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]

        return expr_max + expr_last

In [7]:
def to_pandas(df, cat_cols=None):
    df = df.to_pandas()
    if cat_cols is None:
        cat_cols = list(df.select_dtypes("object").columns)
    df[cat_cols] = df[cat_cols].astype("category")

    return df, cat_cols

def reduce_mem_usage(df):
    """This method reduces memory for numeric columns in the dataframe"""
    numerics = [
        "int16",
        "int32",
        "int64",
        "float16",
        "float32",
        "float64",
        "uint16",
        "uint32",
        "uint64",
    ]
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if "int" in str(col_type):
                if (
                    c_min >= np.iinfo(np.int8).min
                    and c_max < np.iinfo(np.int8).max
                ):
                    df[col] = df[col].astype(np.int8)
                elif (
                    c_min >= np.iinfo(np.int16).min
                    and c_max < np.iinfo(np.int16).max
                ):
                    df[col] = df[col].astype(np.int16)
                elif (
                    c_min >= np.iinfo(np.int32).min
                    and c_max < np.iinfo(np.int32).max
                ):
                    df[col] = df[col].astype(np.int32)
                elif (
                    c_min >= np.iinfo(np.int64).min
                    and c_max < np.iinfo(np.int64).max
                ):
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min >= np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                if (
                    c_min >= np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    print(f"start - end memory:- {start_mem:5.2f} - {end_mem:5.2f} Mb")

    return df

### Read train data

In [8]:
def read_file(path, depth=None):
  df = pl.read_parquet(path)
  df = df.pipe(Pipeline.set_table_dtypes)

  # depth>0 は集計値を使わないと、left join時にOOMになる（RAM 64GBでも)
  if depth in [1, 2]:
    df = df.group_by("case_id").agg(Aggregator.get_exprs(df))

  return df

def read_files(regex_path, depth=None):
  chunks = []
  for path in glob(str(regex_path)):
      df = pl.read_parquet(path)
      df = df.pipe(Pipeline.set_table_dtypes)
      if depth in [1, 2]:
          df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
      chunks.append(df)

  df = pl.concat(chunks, how="vertical_relaxed")
  df = df.unique(subset=["case_id"])

  return df

In [9]:
%%time
train_data_store = {
    "base": read_file(os.path.join(TRAIN_DIR, "train_base.parquet")),
    "depth_0": [
        read_file(os.path.join(TRAIN_DIR, "train_static_cb_0.parquet")),
        read_files(os.path.join(TRAIN_DIR, "train_static_0_*.parquet")),
    ],
    "depth_1": [
        read_files(os.path.join(TRAIN_DIR, "train_applprev_1_*.parquet"), 1),
        read_file(os.path.join(TRAIN_DIR, "train_tax_registry_a_1.parquet"), 1),
        read_file(os.path.join(TRAIN_DIR, "train_tax_registry_b_1.parquet"), 1),
        read_file(os.path.join(TRAIN_DIR, "train_tax_registry_c_1.parquet"), 1),
        read_files(os.path.join(TRAIN_DIR, "train_credit_bureau_a_1_*.parquet"), 1),
        read_file(os.path.join(TRAIN_DIR, "train_credit_bureau_b_1.parquet"), 1),
        read_file(os.path.join(TRAIN_DIR, "train_other_1.parquet"), 1),
        read_file(os.path.join(TRAIN_DIR, "train_person_1.parquet"), 1),
        read_file(os.path.join(TRAIN_DIR, "train_deposit_1.parquet"), 1),
        read_file(os.path.join(TRAIN_DIR, "train_debitcard_1.parquet"), 1),
    ],
    "depth_2": [
        read_file(os.path.join(TRAIN_DIR, "train_credit_bureau_b_2.parquet"), 2),
        read_files(os.path.join(TRAIN_DIR, "train_credit_bureau_a_2_*.parquet"), 2),
        read_file(os.path.join(TRAIN_DIR, "train_applprev_2.parquet"), 2),
        read_file(os.path.join(TRAIN_DIR, "train_person_2.parquet"), 2)
    ],
}

CPU times: user 2min 26s, sys: 26.6 s, total: 2min 52s
Wall time: 16 s


### Feature engineering

In [10]:
def feature_eng(base, depth_0, depth_1, depth_2):
  base_df = (
    base.with_columns(
      month_decision=pl.col("date_decision").dt.month(),
      weekday_decision=pl.col("date_decision").dt.weekday(),
    )
  )

  # 全てのdfをleft join
  for i, df in enumerate(depth_0+depth_1+depth_2):
    base_df = base_df.join(df, how="left", on="case_id", suffix=f"_{i}")

  base_df = base_df.pipe(Pipeline.handle_dates)

  return base_df

In [11]:
train_df = feature_eng(**train_data_store)

del train_data_store
gc.collect()


train_df = train_df.pipe(Pipeline.filter_cols)
train_df, cat_cols = to_pandas(train_df)
train_df = reduce_mem_usage(train_df)

print(f"train data shape: {train_df.shape}")

start - end memory:- 4321.28 - 2224.74 Mb
train data shape: (1526659, 472)


In [12]:
nums = train_df.select_dtypes(exclude="category").columns
nan_df = train_df[nums].isna()

nan_groups = {}
for col in nums:
    cur_group = nan_df[col].sum()
    try:
        nan_groups[cur_group].append(col)
    except:
        nan_groups[cur_group] = [col]

del nan_df
gc.collect()

def reduce_group(grps):
    """
    Reduce the number of columns in each group by selecting the column with the highest number of unique values

    Args:
        grps: list of list of str
    Returns:
        list of str
    """
    use = []
    for g in grps:
        mx = 0
        vx = g[0]
        for gg in g:
            n = train_df[gg].nunique()
            if n > mx:
                mx = n
                vx = gg
        use.append(vx)

    return use

def group_columns_by_correlation(matrix, threshold=0.8):
    """
    Group columns that are highly correlated

    Args:
        - matrix: pd.DataFrame
        - threshold: float
    Returns:
        list of list of str
    """
    correlation_matrix = matrix.corr()
    groups = []
    remaining_cols = list(matrix.columns)
    while remaining_cols:
        col = remaining_cols.pop(0)
        group = [col]
        correlated_cols = [col]
        for c in remaining_cols:
            if correlation_matrix.loc[col, c] > threshold:
                group.append(c)
                correlated_cols.append(c)
        groups.append(group)
        remaining_cols = [c for c in remaining_cols if c not in correlated_cols]

    return groups

uses = []
for k, v in nan_groups.items():
    if len(v) > 1:
        Vs = nan_groups[k]
        grps = group_columns_by_correlation(train_df[Vs], threshold=0.8)
        use = reduce_group(grps)
        uses += use
    else:
        uses += v

uses += list(train_df.select_dtypes("category").columns)
train_df = train_df[uses]

In [13]:
pd.set_option('display.max_columns', 500)
train_df

,case_id,WEEK_NUM,target,month_decision,weekday_decision,credamount_770A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_867L,clientscnt_1022L,clientscnt_100L,clientscnt_1071L,clientscnt_1130L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,deferredmnthsnum_166L,disbursedcredamount_1113A,downpmt_116A,homephncnt_628L,isbidproduct_1095L,mobilephncnt_593L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,sellerplacecnt_915L,max_mainoccupationinc_384A,max_birth_259D,max_num_group1_9,birthdate_574D,dateofbirth_337D,days180_256L,days30_165L,days360_512L,firstquarter_103L,fourthquarter_440L,secondquarter_766L,thirdquarter_1082L,max_debtoutstand_525A,max_debtoverdue_47A,max_refreshdate_3813885D,mean_refreshdate_3813885D,pmtscount_423L,pmtssum_45A,responsedate_1012D,responsedate_4527233D,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,numinstlswithdpd5_4187116L,annuitynextmonth_57A,currdebt_22A,currdebtcredtyperange_828A,numinstls_657L,totalsettled_863A,mindbddpdlast24m_3658935P,avgdbddpdlast3m_4187120P,mindbdtollast24m_4525191P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,maxinstallast24m_3658928A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,maxdpdfrom6mto36m_3546853P,datefirstoffer_1144D,datelastunpaid_3546854D,daysoverduetolerancedd_3976961L,numinsttopaygr_769L,dtlastpmtallstes_4499206D,eir_270L,firstclxcampaign_1125D,firstdatedue_489D,lastactivateddate_801D,lastapplicationdate_877D,mean_creationdate_885D,max_num_group1,max_num_group2_14,last_num_group1_14,last_num_group2_14,lastapprcredamount_781A,lastapprdate_640D,lastdelinqdate_224D,lastrejectcredamount_222A,lastrejectdate_50D,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxdebt4_972A,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdtolerance_374P,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdpdinstldate_3546855D,maxdpdinstlnum_3546846P,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,numinstpaidearly_338L,numinstpaidearly5d_1087L,numinstpaidlate1d_3546852L,numincomingpmts_3546848L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithoutdpd_562L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstregularpaidest_4493210L,numinstpaidearly5dest_4493211L,sumoutstandtotalest_4493215A,numinstpaidlastcontr_4325080L,numinstregularpaid_973L,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sumoutstandtotal_3546847A,totaldebt_9A,mean_actualdpd_943P,max_annuity_853A,mean_annuity_853A,max_credacc_credlmt_575A,max_credamount_590A,max_downpmt_134A,mean_credacc_credlmt_575A,mean_credamount_590A,mean_downpmt_134A,max_currdebt_94A,mean_currdebt_94A,max_mainoccupationinc_437A,mean_mainoccupationinc_437A,mean_maxdpdtolerance_577P,max_outstandingdebt_522A,mean_outstandingdebt_522A,last_actualdpd_943P,last_annuity_853A,last_credacc_credlmt_575A,last_credamount_590A,last_downpmt_134A,last_currdebt_94A,last_mainoccupationinc_437A,last_maxdpdtolerance_577P,last_outstandingdebt_522A,max_approvaldate_319D,mean_approvaldate_319D,max_dateactivated_425D,mean_dateactivated_425D,max_dtlastpmt_581D,mean_dtlastpmt_581D,max_dtlastpmtallstes_3545839D,mean_dtlastpmtallstes_3545839D,max_employedfrom_700D,max_firstnonzeroinstldate_307D,mean_firstnonzeroinstldate_307D,last_approvaldate_319D,last_creationdate_885D,last_dateactivated_425D,last_employedfrom_700D,last_firstnonzeroinstldate_307D,max_byoccupationinc_3656910L,max_childnum_21L,max_pmtnum_8L,last_childnum_21L,last_pmtnum_8L,max

#### Read test data

In [14]:
test_data_store = {
    "base": read_file(os.path.join(TEST_DIR, "test_base.parquet")),
    "depth_0": [
        read_file(os.path.join(TEST_DIR, "test_static_cb_0.parquet")),
        read_files(os.path.join(TEST_DIR, "test_static_0_*.parquet")),
    ],
    "depth_1": [
        read_files(os.path.join(TEST_DIR, "test_applprev_1_*.parquet"), 1),
        read_file(os.path.join(TEST_DIR, "test_tax_registry_a_1.parquet"), 1),
        read_file(os.path.join(TEST_DIR, "test_tax_registry_b_1.parquet"), 1),
        read_file(os.path.join(TEST_DIR, "test_tax_registry_c_1.parquet"), 1),
        read_files(os.path.join(TEST_DIR, "test_credit_bureau_a_1_*.parquet"), 1),
        read_file(os.path.join(TEST_DIR, "test_credit_bureau_b_1.parquet"), 1),
        read_file(os.path.join(TEST_DIR, "test_other_1.parquet"), 1),
        read_file(os.path.join(TEST_DIR, "test_person_1.parquet"), 1),
        read_file(os.path.join(TEST_DIR, "test_deposit_1.parquet"), 1),
        read_file(os.path.join(TEST_DIR, "test_debitcard_1.parquet"), 1),
    ],
    "depth_2": [
        read_file(os.path.join(TEST_DIR, "test_credit_bureau_b_2.parquet"), 2),
        read_files(os.path.join(TEST_DIR, "test_credit_bureau_a_2_*.parquet"), 2),
        read_file(os.path.join(TEST_DIR, "test_applprev_2.parquet"), 2),
        read_file(os.path.join(TEST_DIR, "test_person_2.parquet"), 2)
    ],
}

In [15]:
test_df = feature_eng(**test_data_store)
del test_data_store
gc.collect()

test_df = test_df.select([v for v in train_df.columns if v != "target"])
test_df, cat_cols = to_pandas(test_df, cat_cols) # train時のcat_colsを使用
test_df = reduce_mem_usage(test_df)

start - end memory:-  0.03 -  0.03 Mb


### Feature selection

In [16]:
sample_df: pd.DataFrame = pd.read_csv(os.path.join(ROOT, "sample_submission.csv"))
sample_df = sample_df.set_index("case_id")

device: str = "gpu"
est_cnt: int = 6000

DRY_RUN = True if sample_df.shape[0] == 10 else False
if DRY_RUN:
    device = "cpu"
    train_df = train_df.iloc[:50000]
    est_cnt: int = 600

In [17]:
y = train_df["target"]
weeks = train_df["WEEK_NUM"]

train_df = train_df.drop(columns=["target", "case_id", "WEEK_NUM"])
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

In [18]:
train_df[cat_cols] = train_df[cat_cols].astype(str)
test_df[cat_cols] = test_df[cat_cols].astype(str)

### Train

In [19]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 10,
    "learning_rate": 0.05,
    "n_estimators": 2000,
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    "reg_alpha": 0.1,
    "reg_lambda": 10,
    "extra_trees":True,
    'num_leaves':64,
    "device": device,
    "verbose": -1,
}

In [20]:
params_2 = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 8,
    "learning_rate": 0.03,
    "n_estimators": 2000,
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    "reg_alpha": 0.1,
    "reg_lambda": 10,
    "extra_trees":True,
    'num_leaves':50,
    "device": device,
    "verbose": -1,
}

In [21]:
%%time

fitted_models_cat = []
fitted_models_lgb = []

cv_scores_cat = []
cv_scores_lgb = []

iterator = 0
for idx_train, idx_valid in cv.split(train_df, y, groups=weeks):
    X_train, y_train = train_df.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = train_df.iloc[idx_valid], y.iloc[idx_valid]

    #######
    # cat #
    #######
    train_pool = Pool(X_train, y_train, cat_features=cat_cols)
    val_pool = Pool(X_valid, y_valid, cat_features=cat_cols)

    clf = CatBoostClassifier(
        eval_metric="AUC",
        task_type="GPU",
        learning_rate=0.03,
        iterations=est_cnt,
    )

    clf.fit(train_pool, eval_set=val_pool, verbose=300)
    fitted_models_cat.append(clf)
    y_pred_valid = clf.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores_cat.append(auc_score)

    #######
    # lgb #
    #######
    X_train[cat_cols] = X_train[cat_cols].astype("category")
    X_valid[cat_cols] = X_valid[cat_cols].astype("category")

    if iterator % 2 == 0:
        model = lgb.LGBMClassifier(**params)
    else:
        model = lgb.LGBMClassifier(**params_2)

    model.fit(
        X_train,
        y_train,
        eval_set = [(X_valid, y_valid)],
        callbacks = [lgb.log_evaluation(200), lgb.early_stopping(100)],
    )

    fitted_models_lgb.append(model)
    y_pred_valid = model.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores_lgb.append(auc_score)

    iterator += 1


print("CV AUC scores: ", cv_scores_cat)
print("Maximum CV AUC score: ", max(cv_scores_cat))


print("CV AUC scores: ", cv_scores_lgb)
print("Maximum CV AUC score: ", max(cv_scores_lgb))

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5131940	best: 0.5131940 (0)	total: 46.3ms	remaining: 27.7s
300:	test: 0.7414553	best: 0.7414553 (300)	total: 12.6s	remaining: 12.5s
599:	test: 0.7425879	best: 0.7434287 (555)	total: 25.2s	remaining: 0us
bestTest = 0.7434287369
bestIteration = 555
Shrink model to first 556 iterations.


<timed exec>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.758625
Early stopping, best iteration is:
[105]	valid_0's auc: 0.764815


Default metric period is 5 because AUC is/are not implemented for GPU


...

Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.725446
Early stopping, best iteration is:
[251]	valid_0's auc: 0.726236


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.4968851	best: 0.4968851 (0)	total: 43ms	remaining: 25.8s
300:	test: 0.7241481	best: 0.7241767 (290)	total: 12.6s	remaining: 12.5s
599:	test: 0.7278484	best: 0.7278493 (590)	total: 25.3s	remaining: 0us
bestTest = 0.7278493047
bestIteration = 590
Shrink model to first 591 iterations.


<timed exec>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.738084
Early stopping, best iteration is:
[211]	valid_0's auc: 0.738716
CV AUC scores:  [0.7434291067318943, 0.7877550255337937, 0.7343818432321955, 0.7014894607946911, 0.7278499778768225]
Maximum CV AUC score:  0.7877550255337937
CV AUC scores:  [0.764815416397035, 0.7891029895411878, 0.738460409620672, 0.726235534915644, 0.7387158304825012]
Maximum CV AUC score:  0.7891029895411878
CPU times: user 10min 50s, sys: 9.82 s, total: 11min
Wall time: 2min 34s


### Ensemble

In [22]:
class VotingModel(BaseEstimator, RegressorMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]

        return np.mean(y_preds, axis=0)

    def predict_proba(self, X):
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators[:5]]

        X[cat_cols] = X[cat_cols].astype("category")
        y_preds += [estimator.predict_proba(X) for estimator in self.estimators[5:]]

        return np.mean(y_preds, axis=0)

model = VotingModel(fitted_models_cat+fitted_models_lgb)

### Inference

In [24]:
test_df = test_df.drop(columns=["WEEK_NUM"]).set_index("case_id")
preds = pd.Series(model.predict_proba(test_df)[:, 1], index=test_df.index)
submission_df = pd.DataFrame(index=test_df.index)
submission_df["score"] = preds

submission_df

KeyError: "['WEEK_NUM'] not found in axis"

In [ ]:
submission_df.to_csv("submission.csv")